# importing packages

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# creating function to feed image data to media pipe model

In [2]:
def detect_mediapipe(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# creating function to draw styled landmarks

In [3]:
def draw_styled_landmarks(image, results):

    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

# creating function to calculate distance and to draw a line

In [4]:
def find_distance(image,p1,p2,color):
    
    # drawing a line between points p1 and p2
    cv2.line(image, p1, p2, color, 3)
    
    # taking x, y values from points p1 and p2
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    
    # finding the center
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    
    # finding the length
    length = math.hypot(x2 - x1, y2 - y1)
    
    return length, image, [x1, y1, x2, y2, cx, cy]

# main part of this code where I am calling above created functions

In [5]:
def controller():
    quit = 'Yes'
    # getting devices
    devices = AudioUtilities.GetSpeakers()
    
    # activating above device
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    
    # getting volume range of the selected device
    volRange = volume.GetVolumeRange()
    minVol = volRange[0]
    maxVol = volRange[1]
    
    vol = 0
    volBar = 400
    volPer = 0
    area = 0
    colorVol = (255, 0, 0)

    # capturing live video
    cap = cv2.VideoCapture(0)
    length = 0
    lineInfo =[]
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = detect_mediapipe(frame, holistic)
            
            # getting height and width of the processed image/frame
            h, w, c = image.shape

            # for right hand
            if results.right_hand_landmarks != None:
                
                #for thumb
                x1 = int(results.right_hand_landmarks.landmark[4].x * w )
                y1 = int(results.right_hand_landmarks.landmark[4].y * h )

                #for index finger
                x2 = int(results.right_hand_landmarks.landmark[8].x * w )
                y2 = int(results.right_hand_landmarks.landmark[8].y * h )

                color = (245,117,66)
                point1 = (x1,y1)
                point2 = (x2,y2)

                length, img, lineInfo = find_distance(image,point1,point2,color)

            # for left hand
            if results.left_hand_landmarks != None:

                #for thumb
                x1 = int(results.left_hand_landmarks.landmark[4].x * w )
                y1 = int(results.left_hand_landmarks.landmark[4].y * h )

                #for index finger
                x2 = int(results.left_hand_landmarks.landmark[8].x * w )
                y2 = int(results.left_hand_landmarks.landmark[8].y * h )

                color=(121,22,76)
                point1 = (x1,y1)
                point2 = (x2,y2)

                length, img, lineInfo = find_distance(image,point1,point2,color)

            # Convert Volume
            volBar = np.interp(length, [50, 200], [400, 150])
            volPer = np.interp(length, [50, 200], [0, 100])

            # Reduce Resolution to make it smoother
            smoothness = 10
            volPer = smoothness * round(volPer / smoothness)

            # setting up volume here
            volume.SetMasterVolumeLevelScalar(volPer / 100, None)
            colorVol = (0, 255, 0)
            
            # creating a box to show volume range
            cv2.rectangle(image, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
            
            # putting a text to show volume percentage
            cv2.putText(image, 'Volume '+f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 3)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # Show to screen
            cv2.imshow('Volume', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                print("##                                                   ##")
                print("##               Thankyou for using                  ##")
                print("#######################################################")
                break
                
        cap.release()
        cv2.destroyAllWindows()

In [6]:
print("#######################################################")
print("##                                                   ##")
print("##                   Volume Controller               ##")
print("##                                                   ##")
print("#######################################################")
print("##                                                   ##")
print("##                                                   ##")
print("##                                                   ##")
print("##   Do you want to use this controller?? (yes/no)   ##")
print("##                                                   ##")
print("##                                                   ##")
print("##                                                   ##")
print("#######################################################")

inp = input().lower()
quit = 'no'
if inp in ['yes','y','ye']:
    mp_holistic = mp.solutions.holistic # Holistic model
    mp_drawing = mp.solutions.drawing_utils # Drawing utilities
    quit = 'yes'
    controller()
        


#######################################################
##                                                   ##
##                   Volume Controller               ##
##                                                   ##
#######################################################
##                                                   ##
##                                                   ##
##                                                   ##
##   Do you want to use this controller?? (yes/no)   ##
##                                                   ##
##                                                   ##
##                                                   ##
#######################################################
y
##                                                   ##
##               Thankyou for using                  ##
#######################################################


In [ ]:
quit

In [ ]:
        cap.release()
        cv2.destroyAllWindows()